In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   
from collections import defaultdict
import re
import Levenshtein
from itertools import combinations
from scipy.stats import gmean

os.chdir('C:/Users/Admin/Documents/GitHub/Data-Guide')

In [ ]:
def convert_unix_timestamps(df, column, in_milliseconds=True):
    """
    Convert Unix timestamps in a specified column to datetime.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column (str): The name of the column containing Unix timestamps.
        in_milliseconds (bool): Whether the timestamps are in milliseconds. Default is True.

    Returns:
        pd.DataFrame: The DataFrame with the converted datetime column.
    """
    try:
        factor = 1000 if in_milliseconds else 1
        coll = pd.to_datetime(df[column] / factor, unit='s', errors='coerce')
        print(f"Successfully converted {column} to datetime.")
    except Exception as e:
        coll = df[column]
        print(f"Error converting column {column}: {e}")
    return coll

In [ ]:
def treat_currency(df, column):
    """
    Placeholder for currency treatment logic.
    """
    coll = pd.to_numeric(df[column].replace(r'[\$,]', '', regex=True).replace('-', np.nan), errors='coerce').astype(float)
    coll = coll.fillna(0)
    return coll    

In [ ]:
procedure_map = {
    "Crowns": [
        "Crown - 3/4 porcelain/ceramic", "Full Cast HNM Crown", "Full Porcelain/Ceramic Crown",
        "Implant supported crown - porcelain fused to high noble alloys",
        "Porcelain/HNM Crown", "Porcelain/HNM Pontic", "Porcelain/Noble Crown",
        "Retainer crown - porcelain fused to high noble metal"
    ],
    
    "Prophies": ["Prophylaxis - Adult", "Prophylaxis - Child"],
    
    "Fillings": [
        "Anterior Resin Composite 1s", "Anterior Resin Composite 2s", "Anterior Resin Composite 3s", 
        "Anterior Resin Composite 4+s", "Posterior Resin Composite 1s", "Posterior Resin Composite 2s", 
        "Posterior Resin Composite 3s", "Posterior Resin Composite 4+s"
    ],
    
    "Imaging": [
        "2D Oral/Facial Photo Images", "Bitewing Four Images", "Bitewing Single Image", "Bitewing Two Images",
        "Intraoral â comprehensive series of radiographic images", "Intraoral Periapical Add'l", 
        "Intraoral Periapical Images", "Panoramic Image"
    ],
    
    "Evaluations": [
        "Comprehensive Evaluation", "Periodic Evaluation", "Limited Evaluation", "Re-eval - Post-op Office Visit",
        "Periodontal Evaluation"
    ],
    
    "SRP": ["Scaling & Root Planing (1-3)", "Scaling & Root Planing (4-8)"],
    
    "Perio Maintenance": ["Periodontal Maintenance", "Scaling in presence of generalized gingival inflammation, full mouth"],
    
    "Appliance": [
        "Occlusal guard - hard appliance, full arch", "Orthodontic Retention", 
        "Replacement of lost or broken retainer â mandibular", "Re-cement or re-bond fixed retainer â maxillary",
        "Re-cement or re-bond crown", "Recement/bnd inlay/onlay/part", "Recemnt/bnd cast/prefab pst/cor"
    ],
    
    "Other": [
        "Bone Replacement Graft", "Palliative treatment of dental pain â per visit",
        "Teeth White - In Office", "Teeth White - Take Home", "Topical Applic Fluoride Varnish", 
        "Topical Application of Fluoride", "Sealant", "StellaLife Gel", "StellaLife Rinse",
        "Removal of fixed orthodontic appliances for reasons other than completion of treatment"
    ],
    
    "Scheduling": ["Cancelled Appointment", "Late cancellation fee"],
    
    # ❓ Unsure categories (need clarification)
    "Misc": [
        "Dental Wellness Plan", "Diagnostic/Study Models", "Editorial change to the descriptor", 
        "Misc Invoice", "Routine Extraction ❓ (belongs under surgery?)", 
        "Remove Coronal Remnants - primary tooth ❓", "Limited Occlusal Adjustment ❓",
        "External Bleaching-Office-Arch ❓ (Cosmetic?)"
    ],
    
    "Dentures & Partials": [
        "Interim Lower Partial Denture", "Interim Upper Partial Denture", "Lower Partial w/ Resin Base"
    ],
    
    "Buildups & Adjustments": [
        "Core Buildup w/ Any Pins", "Re-cement or re-bond crown", "Re-cement or re-bond fixed retainer â maxillary"
    ]
}

# Load data

In [ ]:
input_dir = "C:/Users/Admin/Documents/GitHub/Data-Guide/data_pipeline/pull_feb_18" 

output_dir = "C:/Users/Admin/Documents/GitHub/Data-Guide/data_pipeline/transformed_feb_18"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load the data
csv_files = {
        "aged_AR" : os.path.join(input_dir, "aged_ar_report.csv"),
        "statement_submission" : os.path.join(input_dir, "statement_submission_report.csv"),
        "integrated_payments" : os.path.join(input_dir, "integrated_payments_report.csv"),
        #"billing_statement" : os.path.join(input_dir, "billing_statement_report.csv"),
        "outstanding_claims" : os.path.join(input_dir, "outstanding_claims_report.csv"),
        "unresolved_claims" : os.path.join(input_dir, "unresolved_claims_report.csv"),
        #"fee_schedule" : os.path.join(input_dir, "fee_schedule.csv"),
        #"openings" : os.path.join(input_dir,"openings.csv"),
        #"schedule" : os.path.join(input_dir,"schedule.csv"),
        "patient_list" : os.path.join(input_dir, "ZR - Patient List with Details.csv"),
        "processed_payments": os.path.join(input_dir, "ZR - Credit Card Processed Payments.csv"),
        "transaction_details" : os.path.join(input_dir, "ZR - Transaction Detail.csv"),
        "treatment_tracker" : os.path.join(input_dir, "ZR - Treatment Tracker.csv"),
    }

 # Load datasets
dataframes = {dataset: pd.read_csv(file_path) for dataset, file_path in csv_files.items()}

In [ ]:
for d in dataframes.keys():
    print(d)
    print(dataframes[d].columns)
    print("\n")
    print(dataframes[d].head())
    print("\n")

# Merge data

In [ ]:
dataframes['aged_AR']['Ascend Patient ID'] = dataframes['aged_AR']['id']
dataframes['outstanding_claims']['Ascend Patient ID'] = dataframes['outstanding_claims']['patient.id']
dataframes['statement_submission']['Ascend Patient ID'] = dataframes['statement_submission']['patient.id']


for key in dataframes:
    if key not in ['integrated_payments', 'unresolved_claims', 'processed_payments']:
        print(key)
        print(dataframes[key]['Ascend Patient ID'].head())
        df_rows = dataframes[key]['Ascend Patient ID'].apply(lambda x: str(x).isnumeric() and 'Total' not in str(x))
        print(df_rows.head())
        dataframes[key] = dataframes[key].loc[df_rows, :]
        print(dataframes[key].head())
        dataframes[key]['merge_key'] = dataframes[key]['Ascend Patient ID'].astype('Int64').astype(str)

In [ ]:
# Merge the data
dataframes
merged_df = dataframes['patient_list'].copy()
for key in dataframes:
        if key in ['statement_submission', 'aged_AR']:
                print(key)
                merged_df = merged_df.merge(dataframes[key], on='Ascend Patient ID', how='left')

In [ ]:
merged_df.to_csv(f"{output_dir}/merged_data.csv", index=False)

# Transform the Data

In [ ]:
transformed_data = defaultdict(pd.DataFrame)

## Transformations for Aged AR:<br>
- ~~Flag for "remaining guarantor portion"~~
- ~~Total amount and distributions based on bucket - overlaid or grouped~~
- ~~Wide-to-long~~
- - Aging bucket, responsible party, amount

### Row sums

In [ ]:
df_t = dataframes['aged_AR'].copy()

df_t['total_guarantorPortion'] = df_t.loc[:, ['before30.guarantorPortion', 'before60.guarantorPortion', 'before90.guarantorPortion', 'over90.guarantorPortion']].sum(axis=1)
df_t['remaining_guarantorPortion'] = df_t['total_guarantorPortion'] > 0

df_t['total_insurancePortion'] = df_t.loc[:, ['before30.insurancePortion', 'before60.insurancePortion', 'before90.insurancePortion', 'over90.insurancePortion']].sum(axis=1)
df_t['remaining_insurancePortion'] = df_t['total_insurancePortion'] > 0

### Wide to Long

In [ ]:
df_t
df_long = df_t.melt(id_vars=['id', 'Ascend Patient ID'], 
                    value_vars=['before30.amount', 'before60.amount', 'before90.amount', 'over90.amount',
                                'before30.guarantorPortion', 'before60.guarantorPortion', 'before90.guarantorPortion', 'over90.guarantorPortion',
                                'before30.insurancePortion', 'before60.insurancePortion', 'before90.insurancePortion', 'over90.insurancePortion'], 
                    var_name='Aging Bucket', 
                    value_name='Amount')

df_long[['Bucket', 'Responsible Party']] = df_long['Aging Bucket'].str.split('.', expand=True)
df_long['Responsible Party'] = df_long['Responsible Party'].str.replace('amount', 'Total').str.replace('guarantorPortion', 'Guarantor').str.replace('insurancePortion', 'Insurance')
df_long['Bucket'] = df_long['Bucket'].str.replace('before30', '0-30').str.replace('before60', '31-60').str.replace('before90', '61-90').str.replace('over90', '91+')

df_long = df_long.drop(columns=['Aging Bucket'])

print(df_long.head())

In [ ]:
transformed_data['aged_AR'] = df_t.copy()
transformed_data['aged_AR'].to_csv(f"{output_dir}/transformed_aged_AR.csv", index=False)

transformed_data['aged_AR_long'] = df_long.copy()
transformed_data['aged_AR_long'].to_csv(f"{output_dir}/transformed_aged_AR_long.csv", index=False)

## Transformations for Integrated Payments
- ~~Parse transactionCardholderName to pull "discover", "visa", etc~~
- ~~Check "ZZ" for transactionCardholderName~~
- ~~Amount over time~~
- ~~Pair and remove refunds and voids~~

### Rolling Sums

In [ ]:
df_t = dataframes['integrated_payments'].copy()

df_t.head()


In [ ]:

# Rolling Sum

# Ensure the transactionDateTime column is in datetime format
df_t['transactionDateTime'] = convert_unix_timestamps(df_t,'transactionDateTime')

# Set the transactionDateTime as the index
df_t.set_index('transactionDateTime', inplace=True)

# Calculate the rolling sum and count of transactions week-over-week
df_t['rolling_sum'] = df_t['transactionAmount'].rolling('7D').sum()
df_t['transaction_count'] = df_t['transactionAmount'].rolling('7D').count()

df_t['event'] = df_t['ledgerType'].str.replace('CREDIT_CARD_', '').str.lower()

# Reset the index
df_t.reset_index(inplace=True)

print(df_t[['transactionDateTime', 'transactionAmount', 'ledgerType', 'event', 'rolling_sum', 'transaction_count']].head(20))

### Substring extraction

In [ ]:
# Extract card type from transactionCardholderName
df_t['Card Type'] = df_t['transactionCardholderName'].str.extract(
    '(visa|discover|mc|mastercard|amex|americanexpress)', 
    flags=re.IGNORECASE, 
    expand=False
).str.lower()

# Fill missing values with 'unknown'
df_t['Card Type'] = df_t['Card Type'].fillna('unknown')

print(df_t.head(20))

In [ ]:
transformed_data['integrated_payments'] = df_t.copy()
transformed_data['integrated_payments'].to_csv(f"{output_dir}/transformed_integrated_payments.csv", index=False)

## Transformations for Outstanding Claims
- ~~Subscriber DoB > 20 years from Guarantor DoB~~
- Everything by Insurer
- ~~Parse and consolidate group plan name~~
- Balance - Estimate?
- ~~Flag for student plans~~
- ~~- patient DoB vs subscriber DoB~~
- ~~Aggregate insurance carriers that are state-specific~~
- - Blue Cross Blue Shield
- - Delta Dental

In [ ]:
df_t = dataframes['outstanding_claims'].copy()

df_t['student'] = df_t['student'] = (df_t['subscriber.firstName'] != df_t['patient.firstName']) & \
                  (pd.to_datetime(df_t['subscriber.dateOfBirth']) > pd.to_datetime(df_t['patient.dateOfBirth']) + pd.DateOffset(years=18))

df_t['student'] = df_t['student'].replace({True: 'Student', False: 'Non-Student'})

df_t.student.value_counts()

In [ ]:
df_t = dataframes['outstanding_claims'].copy()

In [ ]:
transformed_data['outstanding_claims'] = df_t.copy()
transformed_data['outstanding_claims'].to_csv(f"{output_dir}/transformed_outstanding_claims.csv", index=False)

## Transformations for Patient Details
- ~~Calculate Patient Lifespan~~
- - ~~Today - First Visit and Active~~
- - ~~Last Visit - First Visit~~
- - ~~Next Appointment Date - First Visit~~
- ~~Aggregate insurance carriers that are state-specific~~
- ~~Year of Birth~~
- ~~Month of Birth~~
- ~~Last Visit != Last Procedure~~
- ~~Remove test patients~~
- ~~View uppercase and lowercase patients, empty values~~
- ~~Patient != Guarantor~~
- - ~~Flag for student plans~~
- Parse and aggregate plans
- - Awaiting client response
- ~~Drop "685806 - " from Pat. Prim. Fee Schedule and Discount Plan~~
- Parse address
- ~~Group dates to month~~
- ~~Flag who hasn't been in 6 months~~
- ~~Flag for hasNextAppointment~~
- - ~~and isn't in the past~~
- Flag for Medicare/Medicaid

In [ ]:
df_t = dataframes['patient_list'].copy()

In [ ]:
df_t['Patient'].value_counts()

In [ ]:
# Filter rows where Patient is all uppercase or all lowercase
uppercase_patients = df_t[df_t['Patient'].str.isupper()]
lowercase_patients = df_t[df_t['Patient'].str.islower()]

# Combine the results
filtered_patients = pd.concat([uppercase_patients, lowercase_patients])

print(filtered_patients)
# Isolate patients with "test" in the name or three or more repeated letters
test_patients = df_t[df_t['Patient'].str.lower().str.contains(r'test', case=False, regex=True)]
repeated_letter_patients = df_t[df_t['Patient'].str.contains(r'(.)\1{2,}', case=False, regex=True)]

# Combine the results
patients_to_drop = pd.concat([test_patients, repeated_letter_patients]).drop_duplicates()

print(patients_to_drop)

# Drop the isolated patients from the original dataframe
df_t = df_t[~df_t.index.isin(patients_to_drop.index)]

In [ ]:
df_t['patientAge'] = (pd.to_datetime('today') - pd.to_datetime(df_t['Date Of Birth'])).dt.days // 365.25
df_t['student'] = (df_t['Patient'] != df_t['Primary Guarantor']) & (df_t['patientAge'] < 25)

df_t['student'] = df_t['student'].replace({True: 'Student', False: 'Non-Student'})

df_t.student.value_counts()

In [ ]:
# Ensure the date columns are in datetime format
df_t['First Visit'] = pd.to_datetime(df_t['First Visit'])
df_t['Last Visit'] = pd.to_datetime(df_t['Last Visit'])
df_t['Next Appointment Date'] = pd.to_datetime(df_t['Next Appointment Date'])
df_t['Last Procedure Date'] = pd.to_datetime(df_t['Last Procedure Date'])

# Calculate the patient lifespan
df_t['Lifespan (Today - First Visit)'] = (pd.to_datetime('today') - df_t['First Visit']).dt.days
df_t['Lifespan (Last Visit - First Visit)'] = (df_t['Last Visit'] - df_t['First Visit']).dt.days
df_t['Lifespan (Next Appointment Date - First Visit)'] = (df_t['Next Appointment Date'] - df_t['First Visit']).dt.days

df_t['Time Since Last Visit'] = (pd.to_datetime('today') - df_t['Last Visit']).dt.days
df_t['Visit and Procedure Mismatch'] = df_t['Last Visit'] != df_t['Last Procedure Date']

df_t['hasNextAppointment'] = df_t['Next Appointment Date'].notnull() & (df_t['Next Appointment Date'] > pd.to_datetime('today'))
df_t['overdue'] = (df_t['Time Since Last Visit'] > 183) & (df_t['hasNextAppointment'] == False)

print(df_t[['First Visit', 'Last Visit', 'Last Procedure Date', 'Next Appointment Date', 'Lifespan (Today - First Visit)', 'Lifespan (Last Visit - First Visit)', 'Lifespan (Next Appointment Date - First Visit)', 'Time Since Last Visit',
            'Visit and Procedure Mismatch', 'hasNextAppointment', 'overdue']].head())

In [ ]:
df_t['Pat. Prim. Fee Schedule'] = df_t['Pat. Prim. Fee Schedule'].str.replace('685806 - ', '')
df_t['Discount Plan'] = df_t['Discount Plan'].str.replace('685806 - ', '')

print(df_t[['Pat. Prim. Fee Schedule', 'Discount Plan']].head())

In [ ]:
dataframes['patient_list']['Pat. Prim. Carrier'].value_counts().head(27)

In [ ]:
# Define a function to clean the carrier names
def clean_carrier_name(carrier):
    if pd.isnull(carrier):
        return None
    if isinstance(carrier, str):
        if "Blue Cross Blue Shield" in carrier or "BCBS" in carrier:
            return "Blue Cross Blue Shield"
        elif "Delta" in carrier:
            return "Delta Dental"
        else:
            return re.sub(r' of \w+', '', carrier)
    return carrier

# Extract state information
def extract_state(carrier):
    if pd.isnull(carrier):
        return None
    if isinstance(carrier, str):
        if "Blue Cross Blue Shield" or "BCBS" in carrier:
            match = re.search(r' of (\w+)', carrier)
            if not match:
                match = re.search(r'\((\w{2}(?: \w{2})+)\)', carrier)
                if match:
                    return match.group(1)
        elif "Delta Dental" in carrier:
            match = re.search(r' of (\w+)', carrier)
        else:
            return re.sub(r' of \w+', '', carrier)
        match = re.search(r' of (\w+)', carrier)
        return match.group(1) if match else None
    return carrier

# Apply the function to create a new column
df_t['Affiliate State'] = df_t['Pat. Prim. Carrier'].apply(extract_state)

# Apply the function to create a new column
df_t['Cleaned Carrier'] = df_t['Pat. Prim. Carrier'].apply(clean_carrier_name)

print(df_t[['Pat. Prim. Carrier', 'Cleaned Carrier', 'Affiliate State']].head(20))

In [ ]:
#!pip install fuzzywuzzy

from fuzzywuzzy import process, fuzz
import itertools

#Extract unique values from the 'Pat. Prim. Plan' column
plans = df_t['Pat. Prim. Plan'].dropna().unique()

#Calculate similarity scores for all pairs
similarity_scores = []
for plan1, plan2 in itertools.combinations(plans, 2):
    score = fuzz.ratio(plan1, plan2)
    similarity_scores.append((plan1, plan2, score))

#Sort the pairs by similarity score in descending order
similarity_scores.sort(key=lambda x: x[2], reverse=True)

#Convert similarity scores to a DataFrame
similarity_df = pd.DataFrame(similarity_scores, columns=['Plan 1', 'Plan 2', 'Similarity Score'])

#Save the DataFrame to a CSV file
similarity_df.to_csv("similarity_scores.csv", index=False)

In [ ]:
df_t.head()

In [ ]:
# Process the address data
parsed_data = []

addresses = df_t['Address']

for address in addresses:
    parts = [p.strip() for p in address.split(",")]  # Split by comma and remove extra spaces
    
    # Handle different address structures dynamically
    street = parts[0] if len(parts) > 0 else ""
    apartment = parts[1] if len(parts) > 3 else ""  # If there's a second part but before city/state
    city = parts[-3].lower() if len(parts) > 2 else ""  # City is the third-to-last part
    state = parts[-2] if len(parts) > 1 else ""  # State is the second-to-last part
    zip_code = parts[-1] if len(parts) > 0 else ""  # ZIP is always last

    parsed_data.append({
        "Street": street,
        #"Apartment": apartment,
        "City": city,
        "State": state,
        "ZIP Code": zip_code
    })

# Convert to DataFrame for display
df_addresses = pd.DataFrame(parsed_data)

df_addresses.head(20)
# Add parsed columns back to the original dataframe
df_t = pd.concat([df_t.reset_index(drop=True), df_addresses.reset_index(drop=True)], axis=1)

In [ ]:
df_t.head()

In [ ]:
df_t.columns

In [ ]:
df_t['DCS'] = df_t['Pat. Prim. Plan'].str.contains('DCS')
df_t.groupby('Status')['DCS'].value_counts()

In [ ]:
df_t['DNU'] = df_t['Pat. Prim. Plan'].str.contains('DNU')
df_t.groupby('Status')['DNU'].value_counts()

In [ ]:
# # Concatenate DCS and DNU columns
# df_t['DCS_DNU'] = df_t['DCS'].astype(str) + '_' + df_t['DNU'].astype(str)

# # Get the count of combinations for Status and DCS_DNU
# status_dcs_dnu_counts = df_t.groupby(['Status', 'DCS_DNU']).size().reset_index(name='Count')

# print(status_dcs_dnu_counts)

In [ ]:
def clean_plan_name(plan_name):
    """
    Cleans a dental plan name by removing specified substrings and formatting it properly.

    Args:
        plan_name (str): The original plan name.

    Returns:
        str: The cleaned plan name.
    """
    if not isinstance(plan_name, str) or not plan_name.strip():
        return plan_name  # Return as is if empty or not a string

    # Substrings to remove (case insensitive)
    remove_substrings = ["inc", "llc", "dcs", "dnu", "corporation", "incorporated", "technologies", "healthcare", "international", "partners", "systems", "services"]

    # Remove specified substrings
    for substring in remove_substrings:
        plan_name = re.sub(rf"\b{substring}\b", "", plan_name, flags=re.IGNORECASE)

    # Replace non-alphanumeric characters with a space
    plan_name = re.sub(r"[^a-zA-Z0-9]", " ", plan_name)

    # Trim multiple spaces down to a single space
    plan_name = re.sub(r"\s+", " ", plan_name).strip()

    return plan_name

In [ ]:
df_t['plan_name'] = df_t['Pat. Prim. Plan'].apply(clean_plan_name)

In [ ]:
def extract_non_overlapping_parts(plan, cleaned_plan):
    """
    Extract non-overlapping parts and words containing non-alphabetic characters from the original plan name.

    Args:
        plan (str): The original plan name.
        cleaned_plan (str): The cleaned plan name.

    Returns:
        str: A string containing the non-overlapping parts and words with non-alphabetic characters.
    """
    if not isinstance(plan, str) or not isinstance(cleaned_plan, str):
        return plan  # Return as is if not a string

    # Substrings to remove (case insensitive)
    remove_substrings = ["inc", "llc", "dcs", "dnu", "corporation", "incorporated", "technologies", "healthcare", "international", "partners", "systems", "services"]
    plan = re.sub(r'[.,-]', '', plan)  # Remove commas, periods, and hyphens

    # Remove specified substrings
    for substring in remove_substrings:
        plan = re.sub(rf"\b{substring}\b", "", plan, flags=re.IGNORECASE)

    # Split the original and cleaned plan names into words
    plan_words = set(plan.split())
    cleaned_plan_words = set(cleaned_plan.split())

    # Find non-overlapping words
    non_overlapping_words = plan_words - cleaned_plan_words

    # Find words containing non-alphabetic characters
    words_with_non_alpha = {word for word in plan_words if not word.isalpha()}

    # Combine the results
    result = non_overlapping_words.union(words_with_non_alpha)

    # Remove empty strings, words with ending hyphens, and single characters

    # Join the results and strip consecutive whitespace to single spaces
    return ' '.join(result).strip()


In [ ]:
# Apply the function to create a new column
df_t['plan_name_cruft'] = df_t.apply(lambda row: extract_non_overlapping_parts(row['Pat. Prim. Plan'], row['plan_name']), axis=1)

# Display the results
print(df_t[['Pat. Prim. Plan', 'plan_name', 'plan_name_cruft']].head())

In [ ]:
df_t['plan_name_cruft'].value_counts().head(60)

In [ ]:
test_case_ind = df_t['plan_name_cruft'].str.contains("SCH/CAREERCTU", regex=False).fillna(False)
df_t.loc[test_case_ind,['Pat. Prim. Plan', 'plan_name', 'plan_name_cruft']]

In [ ]:
test_case_ind = df_t['plan_name'].str.lower().str.contains("ass", regex=False).fillna(False)
df_t.loc[test_case_ind,['Pat. Prim. Plan', 'plan_name', 'plan_name_cruft']]

In [ ]:
test_case_ind = df_t['Pat. Prim. Plan'].str.contains("LENOVO (UNITED STATES) INC. (2500)- DCS DNU", regex=False).fillna(False)
df_t.loc[test_case_ind,:]

In [ ]:
# Define invalid substring pairs (e.g., "HMO" should not match "PPO")
INVALID_SUBSTRING_PAIRS = [
    ("HMO", "PPO"), 
    ("Advantage", "Medicaid"), 
    ("Basic", "Premium"), 
    ("Select", "Standard"),
    ("Select", "Basic"),
    ("(Self)", "(Self)"),
    ("1A", "1B"),
    ("Teacher", "Career"),
    ('Diamond', 'Core'),
    ('City of', 'University of'),
]


In [ ]:
def longest_common_substring(str1, str2):
    """
    Find the longest common substring between two strings using dynamic programming.

    Args:
        str1 (str): First string.
        str2 (str): Second string.

    Returns:
        int: Length of the longest common substring.
    """
    str1, str2 = str1.lower(), str2.lower()
    len1, len2 = len(str1), len(str2)
    dp = np.zeros((len1 + 1, len2 + 1), dtype=int)
    max_length = 0

    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            if str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
                max_length = max(max_length, dp[i][j])

    return max_length

def substring_overlap_percentage(str1, str2):
    """
    Compute the longest common substring overlap as a percentage of the shortest string.
    Also includes a Levenshtein similarity check.

    Args:
        str1 (str): First string.
        str2 (str): Second string.

    Returns:
        dict: Overlap percentage, Levenshtein similarity, and geometric mean.
    """
    if not str1 or not str2:
        return {"overlap_percentage": 0.0, "levenshtein_similarity": 0.0, "geometric_mean": 0.0}

    str1, str2 = str1.lower(), str2.lower()
    longest_substring = longest_common_substring(str1, str2)
    min_length = min(len(str1), len(str2))

    overlap_percentage = (longest_substring / min_length) * 100 if min_length > 0 else 0.0
    levenshtein_sim = 1 - (Levenshtein.distance(str1, str2) / max(len(str1), len(str2)))

    similarity_scores = [overlap_percentage, levenshtein_sim * 100]
    geometric_mean_value = gmean(similarity_scores) if min(similarity_scores) > 0 else 0.0

    return {
        "overlap_percentage": round(overlap_percentage, 2),
        "levenshtein_similarity": round(levenshtein_sim * 100, 2),
        "geometric_mean": round(geometric_mean_value, 2)
    }

def contains_invalid_substring_pair(str1, str2):
    """
    Check if a string pair contains any invalid substring combinations.

    Args:
        str1 (str): First string.
        str2 (str): Second string.

    Returns:
        bool: True if an invalid pair is found, otherwise False.
    """
    for substr1, substr2 in INVALID_SUBSTRING_PAIRS:
        if (substr1.lower() in str1.lower() and substr2.lower() in str2.lower()) or \
           (substr2.lower() in str1.lower() and substr1.lower() in str2.lower()):
            return True
    return False

def batch_process_similarity(df, column_name):
    """
    Compute substring overlap, Levenshtein similarity, and geometric mean for all unique string pairs.

    Args:
        df (pd.DataFrame): DataFrame containing a column of text values.
        column_name (str): Column name containing the text data.

    Returns:
        pd.DataFrame: DataFrame with pairwise similarity scores and an invalid pair flag.
    """
    unique_strings = df[column_name].dropna().unique()
    string_pairs = list(combinations(unique_strings, 2))

    results = []
    for str1, str2 in string_pairs:
        invalid_pair = contains_invalid_substring_pair(str1, str2)
        scores = substring_overlap_percentage(str1, str2)
        results.append({
            "string_1": str1,
            "string_2": str2,
            **scores,
            "invalid_pair": invalid_pair  # Flag invalid pairs instead of skipping them
        })

    return pd.DataFrame(results)


In [ ]:
similarity_results = batch_process_similarity(df_t, "plan_name")


In [ ]:
similarity_results.sort_values("geometric_mean", ascending=False).head(20)

In [ ]:

# Save results
similarity_results.to_csv(f"{output_dir}/plan_similarity_results.csv", index=False)

# Display output
print(similarity_results)


In [ ]:
df_t['university_or_college_plan'] = df_t['Pat. Prim. Plan'].str.contains('university' or 'college' or 'u of', case=False, na=False)
df_t['university_or_college_plan'].value_counts()

In [ ]:
consolidate_plans = pd.read_csv(f"{output_dir}/plan_consolidation_v2.csv")

In [ ]:
import networkx as nx

# Ensure columns are properly named (assuming first two columns represent equivalent plans)
consolidate_plans.columns = ["Plan_A", "Plan_B"]

# Create a graph to model the equivalence relationships
G = nx.Graph()
G.add_edges_from(consolidate_plans.values)

# Find connected components (each component represents a set of equivalent plans)
components = list(nx.connected_components(G))

components

In [ ]:
# Create a mapping from each plan to the simplest representation (sorted alphabetically within each component)
equivalence_mapping = {}
for component in components:
    sorted_plans = sorted(component, key=lambda x: (len(x), x))  # Sort by length and then alphabetically
    simplest_plan = sorted_plans[0].lower()  # Choose the first (simplest) plan name
    for plan in component:
        equivalence_mapping[plan] = simplest_plan

# Convert the mapping into a DataFrame for easy inspection
mapping_df = pd.DataFrame(list(equivalence_mapping.items()), columns=["Plan_Name", "Simplest_Equivalent"])

mapping_df.head(20)

In [ ]:
df_t['Simplest_Equivalent_Plan'] = df_t['plan_name'].map(mapping_df.set_index('Plan_Name')['Simplest_Equivalent']).fillna(df_t['plan_name'].str.lower())

In [ ]:
df_t['Simplest_Equivalent_Plan'].value_counts().head(50)

In [ ]:
df_t['Pat. Prim. Plan'].value_counts().head(28)

In [ ]:
# Apply the function to create a new column
df_t['temp'] = df_t['plan_name'].str.lower()
df_t['plan_name_diff'] = df_t.apply(lambda row: extract_non_overlapping_parts(row['temp'], row['Simplest_Equivalent_Plan']), axis=1)

df_t.drop(columns=['temp'], inplace=True)

# Display the results
print(df_t[['Pat. Prim. Plan', 'plan_name', 'DCS', 'DNU', 'Simplest_Equivalent_Plan', 'plan_name_diff']].head(50))

In [ ]:
df_t.head(50)

In [ ]:
transformed_data['patient_details'] = df_t.copy()
transformed_data['patient_details'].to_csv(f"{output_dir}/transformed_patient_details.csv", index=False)

In [ ]:
transformed_data['active_patients'] = df_t.loc[df_t['Status'] == 'ACTIVE',:].copy()
transformed_data['active_patients'].to_csv(f"{output_dir}/transformed_active_patient_details.csv", index=False)

## Transformations for Processed Payments
- ~~Break out and parse transaction types~~
- ~~Parse Reference Number to pull out Aenta, DD, Cigna, etc~~
- Split dataset into Insurance, Patient, Office, etc
- Group by card type, insurance, etc
- Amount by date
- ~~Calculate differences between Not Available dates and other dates to get time to various metrics, like time to full payment~~
- ~~Calculate running totals ~~
- Maybe make dataset that's more of as point-in-time option
- ~~Flip negative and positive~~

In [ ]:
df_t = dataframes['processed_payments'].copy()

In [ ]:
df_t.head()

In [ ]:
df_t['Amount'] = df_t['Amount'].str.replace('$', '').str.replace(',', '').astype(float) * -1

In [ ]:
matches = df_t['Reference Number'].str.contains(r"(?i)test", na=False, regex=True)
df_t[matches == True]

In [ ]:
# Drop "test" rows
df_t = df_t[matches != True]

In [ ]:
# Ensure the Date (Modified) column is in datetime format
df_t['Date (Modified)'] = pd.to_datetime(df_t['Date (Modified)'])

# Create a sort key for the Transaction Type
df_t['sort_key'] = 1
df_t.loc[df_t['Transaction Type'] == 'Not Available', 'sort_key'] = 0
#df_t['sort_key'] = df_t['Transaction Type'].replace({'Not Available': 0, 'Payment': 1, 'Adjustment': 2, 'Refund': 3})

# Sort the dataset by Ascend Patient ID and Date (Modified)
df_t.sort_values(by=['Ascend Patient ID', 'Date (Modified)', 'sort_key'], inplace=True)

# Calculate the difference in days between consecutive dates for each Ascend Patient ID
df_t['Days Between'] = df_t.groupby('Ascend Patient ID')['Date (Modified)'].diff().dt.days
# Initialize a new column for the rolling difference
df_t['Rolling Days Between'] = np.nan

# Iterate through each Ascend Patient ID
for patient_id, group in df_t.groupby('Ascend Patient ID'):
    # Sort the group by Date (Modified)
    group = group.sort_values(by=['Date (Modified)', 'sort_key'])
    
    # Initialize the last incurred charge date
    last_incurred_date = None
    
    # Iterate through each row in the group
    for idx, row in group.iterrows():
        if row['Transaction Type'] == 'Not Available':
            # Update the last incurred charge date
            last_incurred_date = row['Date (Modified)']
            df_t.loc[idx, 'Rolling Days Between'] = 0
        elif last_incurred_date is not None:
            # Calculate the difference in days from the last incurred charge date
            df_t.loc[idx, 'Rolling Days Between'] = (row['Date (Modified)'] - last_incurred_date).days
        print(last_incurred_date)

# Filter the dataset to calculate the days between dates for "Transaction Type" = "Not Available" and other transaction types
df_not_available = df_t[df_t['Transaction Type'] == 'Not Available']
df_other = df_t[df_t['Transaction Type'] != 'Not Available']

# Display the results
#print(df_not_available[['Ascend Patient ID', 'Transaction Type', 'Date (Modified)', 'Days Between', 'Rolling Days Between']])
#print(df_other[['Ascend Patient ID', 'Transaction Type', 'Date (Modified)', 'Days Between', 'Rolling Days Between']])

In [ ]:
#df_t['Transaction Type'].value_counts()

In [ ]:
df_t[['Ascend Patient ID', 'Transaction Type', 'Date (Modified)', 'Days Between', 'Rolling Days Between']].head(28)

In [ ]:
# Ensure the Amount column is in numeric format
df_t['Amount'] = df_t['Amount'].astype(float)

# Calculate the running total for Amount by each Ascend Patient ID
df_t['Running Total'] = df_t.groupby('Ascend Patient ID')['Amount'].cumsum()

# Display the results
print(df_t[['Ascend Patient ID', 'Transaction Type', 'Date (Modified)', 'Amount', 'Running Total']].head(20))

In [ ]:
df_t1 = df_t.copy()
df_t2 = df_t.copy()

df_t1 = df_t1.loc[df_t1['Transaction Type'] != 'Not Available']
df_t2 = df_t2.loc[df_t2['Transaction Type'] == 'Not Available']

In [ ]:
# Extract card type from transactionCardholderName
df_t1['Extracted Card Type'] = df_t1['Card Holder'].str.extract(
    '(visa|discover|mc|mastercard|amex|americanexpress)', 
    flags=re.IGNORECASE, 
    expand=False
).str.lower()

# Fill missing values with 'unknown'
df_t1['Extracted Card Type'] = df_t1['Extracted Card Type'].fillna('unknown')

In [ ]:
df_t.dtypes

In [ ]:
#dataframes['patient_list']['Pat. Prim. Carrier'].value_counts().head(28)

In [ ]:
# Extract card type from transactionCardholderName
#test = df_t1['Reference Number'].str.replace(r'\d+', '', regex=True).str.lower()

#test.value_counts().head(28)

In [ ]:
# Extract card type from transactionCardholderName
df_t1['Extracted Insurance'] = df_t1['Reference Number'].str.replace(r'\d+', '', regex=True).str.lower().str.extract(
    r'^(cigna|dd|mc|aetna|ml|uhc|sl|guardian|geha)', 
    flags=re.IGNORECASE, 
    expand=False
)

# Fill missing values with 'unknown'
df_t1['Extracted Insurance'] = df_t1['Extracted Insurance'].fillna('unknown')

df_t1['Extracted Insurance'].value_counts()

In [ ]:
transformed_data['processed_payments'] = df_t.copy()
transformed_data['payments'] = df_t1.copy()
transformed_data['incurred_charges'] = df_t2.copy()

transformed_data['processed_payments'].to_csv(f"{output_dir}/transformed_processed_payments.csv", index=False)
transformed_data['payments'].to_csv(f"{output_dir}/transformed_payments.csv", index=False)
transformed_data['incurred_charges'].to_csv(f"{output_dir}/transformed_incurred_charges.csv", index=False)

## Transformations for Statement Submissions
- ~~Drop or flag negative balance~~
- Grouping by generatedFrom, source, statement type, month
- Balance over time total
- ~~Balance over time - patient~~

In [ ]:
df_t = dataframes['statement_submission'].copy()

In [ ]:
matches = (df_t['patient.firstName'] + ", " + df_t['patient.lastName']).str.contains(r"(?i)test", na=False, regex=True)
df_t[matches == True]

In [ ]:
# Drop "test" rows
df_t = df_t[matches != True]

In [ ]:
df_t['dateTime'] = convert_unix_timestamps(df_t, 'dateTime')

In [ ]:
df_t['neg_balances'] = df_t['balance']<0
df_t.loc[df_t['neg_balances'],:].head()

In [ ]:
df_t.dtypes

In [ ]:
# Ensure the balance column is in numeric format
df_t['balance'] = pd.to_numeric(df_t['balance'], errors='coerce')

# Ensure the patient.id column is in string format
df_t['patient.id'] = df_t['patient.id'].astype(str)

# Ensure the dateTime column is in datetime format
df_t['dateTime'] = pd.to_datetime(df_t['dateTime'], errors='coerce')

# Sort the dataframe by patient.id and dateTime
df_t = df_t.sort_values(by=['patient.id', 'dateTime'])

# Calculate the marginal difference between adjacent rows on the "balance" column
df_t['balance_diff'] = df_t.groupby('patient.id')['balance'].diff()

# Display the results
print(df_t[['patient.id', 'dateTime', 'balance', 'balance_diff']].head(20))

In [ ]:
# Ensure the dateTime column is in datetime format
df_t['dateTime'] = pd.to_datetime(df_t['dateTime'], errors='coerce')

# Sort the dataframe by patient.id and dateTime
df_t = df_t.sort_values(by=['patient.id', 'dateTime'])

# Add a column for possible duplicates based on adjacent dates
df_t['possibleDuplicate'] = df_t.duplicated(subset=['patient.id', 'balance'], keep=False) & \
                            df_t.duplicated(subset=['patient.id', 'balance'], keep='last').shift(1).fillna(False)

# Display the results
print(df_t.head(20))

In [ ]:
transformed_data['statement_submission'] = df_t.copy()
transformed_data['statement_submission'].to_csv(f"{output_dir}/transformed_statement_submission.csv", index=False)

## Tranformations for Transaction Details
- Charge by Category
- Categorical Comparisons
- Charges by date
- Possibly split based on Category
- ~~Flag for referrals based on Proc Code~~
- ~~Parse substrings on Proc Description~~
- ~~Flag negative Charges~~

In [ ]:
df_t = dataframes['transaction_details'].copy()
df_t.head()

In [ ]:
df_t['Charges'] = treat_currency(df_t, 'Charges')
df_t['Credits'] = treat_currency(df_t, 'Credits')

In [ ]:
df_t['neg_charges'] = df_t['Charges']<0
df_t.loc[df_t['neg_charges'],:].head()

In [ ]:
df_t['neg_charges'].value_counts()

In [ ]:
df_t['pos_credits'] = df_t['Credits']>0
df_t.loc[df_t['pos_credits'],:].head()

In [ ]:
df_t['pos_credits'].value_counts()

In [ ]:
from collections import Counter
import itertools

# Extract all words from the 'Proc Description' column
words = df_t['Proc. Description'].str.lower().str.split().explode()

# Count the frequency of each word
word_counts = Counter(words)

# Find common substrings of whole words
common_words = {word: count for word, count in word_counts.items() if count > 1}

# Sort the common words by their frequency in descending order
sorted_common_words = dict(sorted(common_words.items(), key=lambda item: item[1], reverse=True))

print(sorted_common_words)
# Plot the top 24 words
top_24_words = dict(itertools.islice({word: count for word, count in sorted_common_words.items() if len(word) > 4}.items(), 24))

plt.figure(figsize=(12, 8))
plt.bar(top_24_words.keys(), top_24_words.values())
plt.xticks(rotation=90)
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.title('Top 24 Most Common Words in Procedure Descriptions')
plt.show()

In [ ]:
phrase_list = ['periodontal', 'evaluation', 'intraoral', 'periapical', 'prophylaxis', 'bitewing', 'resin', 'composite', 'porcelain', 'ceramic', 'images', 'arch', 'guard', 'gingival', 'fluoride', 'root', 'crown', 'buildup',
               'metal', 'replacement', 'noble', 'upper', 'lower', 'denture', 'retainer', 'medicaments','palliative', 'late', 'cancellation', 'orthodontic', 'extraction', 'posterior','anterior', 'panoramic', 'radiographic']

In [ ]:
# Initialize the count columns for each word in the word list
for word in phrase_list:
    df_t[word] = df_t['Proc. Description'].str.contains(word, case=False, regex=False).astype(bool)

# Display the updated dataframe
print(df_t.head())

In [ ]:
df_t.dtypes

In [ ]:
# Calculate the correlation matrix
df_t_corr = df_t.copy()
df_t_corr = df_t_corr.drop(columns=['Charges', 'Credits', 'neg_charges', 'pos_credits', 'Patient', 'Proc. Description', 'Date', 'Ascend Patient ID', 'Chart Number', 'merge_key'])

df_t_corr_num = df_t_corr.select_dtypes(include=[np.number]).copy()
correlation_matrix = df_t_corr_num.corr()

# Find pairs of columns that are perfectly correlated
perfect_correlations = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if correlation_matrix.iloc[i, j] == 1.0:
            colname1 = correlation_matrix.columns[i]
            colname2 = correlation_matrix.columns[j]
            perfect_correlations.append((colname1, colname2))
    # Check for perfect binary or categorical relationships
for column in df_t_corr.select_dtypes(include=['object', 'category', 'bool']).columns:
    unique_values = df_t_corr[column].nunique()
    if unique_values == 2:
        for other_column in df_t_corr.select_dtypes(include=['object', 'category', 'bool']).columns:
            if column != other_column and df_t_corr.groupby(column)[other_column].nunique().max() == 1:
                perfect_correlations.append((column, other_column))

    # Print the pairs of perfectly correlated columns including binary or categorical relationships
    print("Perfectly correlated column pairs including binary or categorical relationships:")
    for pair in perfect_correlations:
        print(pair)

In [ ]:
df_t['Referral'] = df_t['Proc. Code'].str.startswith('REF')
df_t['Referral'].value_counts()

In [ ]:
transformed_data['transaction_details'] = df_t.copy()
transformed_data['transaction_details'].to_csv(f"{output_dir}/transformed_transaction_details.csv", index=False)

## Transformations for Treatment Tracker
- Add month grouping
- Remove test patients
- Remove canceled and/or invalidated (?)

In [ ]:
df_t = dataframes['treatment_tracker'].copy()

In [ ]:
transformed_data['treatment_tracker'] = df_t.copy()

## Transformations for Unresolved Claims
- Plans by carrier grouping
- Parse StateId
- Group by: Carrier, Plan

In [ ]:
df_t = dataframes['unresolved_claims'].copy()

In [ ]:
transformed_data['unresolved_claims'] = df_t.copy()

# Merge transformed data

In [ ]:
# Merge the data
transformed_data
merged_df = transformed_data['patient_details'].copy()
for key in transformed_data:
        if key in ['statement_submission']:
                print(key)
                merged_df = merged_df.merge(transformed_data[key], on='Ascend Patient ID', how='left')

In [ ]:
merged_df.to_csv(f"{output_dir}/merged_data.csv", index=False)

In [ ]:
merged_df.head

In [ ]:
merged_df.columns

In [ ]:
transformed_data['patient_details'].dtypes